<center> <h1 style="background-color:indianred; color:white" >Pima Indians Diabetes Classification & Comparision</h1> 

<center><img
src="https://upload.wikimedia.org/wikipedia/commons/6/6f/Pima.jpg" style="width:30%;height:40%;">
</center>
    
<br>    
    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:indianred; color:white" data-toggle="list"  role="tab" aria-controls="home">Notebook Content!</h3>  
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#section1" role="tab" aria-controls="profile" style="color:indianred">Import Libraries<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">1</span></a>    
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#section2" role="tab" aria-controls="profile" style="color:indianred">Load Data<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">2</span></a>
  <a id="section2" class="list-group-item list-group-item-action" data-toggle="list" href="#section3" role="tab" aria-controls="messages" style="color:indianred">Missing Values<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">3</span></a>
  <a class="list-group-item list-group-item-action"  data-toggle="list" href="#section4" role="tab" aria-controls="settings" style="color:indianred">Filling Missing Values and Creating New Variables<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">4</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#section5" role="tab" aria-controls="settings" style="color:indianred">New Features<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">5</span></a> 
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#section6" role="tab" aria-controls="settings" style="color:indianred">Numeric Variables<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">6</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#section7" role="tab" aria-controls="settings" style="color:indianred">Categoric Variables<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">7</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#section8" role="tab" aria-controls="settings" style="color:indianred">One-Hot Encoder<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">8</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#section9" role="tab" aria-controls="settings" style="color:indianred">Model Comparision<span class="badge badge-primary badge-pill" style="background-color:indianred; color:white">9</span></a>

<a id="section1"></a>
# 1. Packages

In [ ]:
# Base 
# -----------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Models 
# -----------------------------------
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# Metrics & Evaluation
# -----------------------------------
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report

# Configuration
# -----------------------------------
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

<a id="section2"></a>
# 2. Data

In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
df.head()

- **Pregnancies:** Number of times pregnant
- **Glucose:** Plasma glucose concentration a 2 hours in an oral glucose tolerance test
- **BloodPressure:** Diastolic blood pressure (mm Hg)
- **SkinThickness:** Triceps skin fold thickness (mm)
- **Insulin:** 2-Hour serum insulin (mu U/ml)
- **BMI:** Body mass index (weight in kg/(height in m)^2)
- **DiabetesPedigreeFunction:** Diabetes pedigree function
- **Age:** Age (years)
- **Outcome:** Class variable (0 or 1)

In [ ]:
df.describe([0.01, 0.05, 0.10, 0.20, 0.7, 0.8, 0.9, 0.95, 0.99]).T

In [ ]:
# Target
print(df.Outcome.value_counts())
print(df.Outcome.value_counts() / df.shape[0])

<a id="section3"></a>
# 3. Missing Values

In [ ]:
print(df.isnull().sum() / df.shape[0], "\n")

missing = ["Glucose", "BMI", "BloodPressure", "SkinThickness", "Insulin"]
for i in missing:
    df[i] = np.where(df[i] == 0, np.nan, df[i])
    
def missing_imp_category(data, group_variable, target, method = "mean"):
    data[target] = data[target].fillna(data.groupby(group_variable)[target].transform(method))
    
df.isnull().sum().sort_values(ascending = False) / df.shape[0]

<a id="section4"></a>
# 4. Filling Missing Values and Creating New Variables

In [ ]:
# Age Category
df["AGECAT"] = pd.cut(df.Age, 5, labels = np.arange(1,6,1))

# Fill Missing
missing_imp_category(df, group_variable=["AGECAT", "Outcome"], target = "BMI", method="median")

<center><img
src="https://www.cdc.gov/healthyweight/images/assessing/bmi-adult-fb-600x315.jpg" style="width:45%;height:40%;">
</center>

In [ ]:
# BMI Category
df["BMI_CAT"] = np.where(df.BMI < 18.5, "UNDERWEIGHT", "NORMAL")
df["BMI_CAT"] = np.where((df.BMI >= 25) & (df.BMI < 30), "OVERRWEIGHT", df["BMI_CAT"])
df["BMI_CAT"] = np.where((df.BMI >= 30) & (df.BMI < 35), "OBESE", df["BMI_CAT"])
df["BMI_CAT"] = np.where((df.BMI >= 35), "EXTREMELY OBESE", df["BMI_CAT"])
df["ISOBESE"] = np.where(df.BMI >= 30, 1, 0)

# Fill Missing
missing_imp_category(df, group_variable=["BMI_CAT", "Outcome"], target = "SkinThickness", method="median")

# Fill Missing
df["Glucose"] = df.Glucose.fillna(df.Glucose.median())

<center><img
src="https://img.webmd.com/dtmcms/live/webmd/consumer_assets/site_images/article_thumbnails/other/blood_pressure_charts/basic_blood_pressure_chart.png" style="width:45%;height:40%;">
</center>



In [ ]:
# Fill Missing
missing_imp_category(df, group_variable=["AGECAT", "Outcome"], target = "BloodPressure", method="median")

# DIASTOLIC
df["DIASTOLIC"] = np.where(df.BloodPressure < 80, "Normal", "Hypertension Stg1")
df["DIASTOLIC"] = np.where((df.BloodPressure >= 90) & (df.BloodPressure < 120), "Hypertension Stg2", df["DIASTOLIC"])
df["DIASTOLIC"] = np.where((df.BloodPressure > 120), "Hypertensive Crisis", df["DIASTOLIC"])


# Fill Missing
missing_imp_category(df, group_variable=["AGECAT", "Outcome"], target = "Insulin", method="median")
missing_imp_category(df, group_variable=["Outcome"], target = "Insulin", method="median")

<a id="section5"></a>
# 5. New Features

In [ ]:
# Pregnancies
df["CHILD"] = np.where(df.Pregnancies > 0, 1, 0)
df["PregnanciesCAT"] = np.where(df.Pregnancies == 0, "No", "1-3")
df["PregnanciesCAT"] = np.where((df.Pregnancies > 3) & (df.Pregnancies < 10), "4-9", df["PregnanciesCAT"])
df["PregnanciesCAT"] = np.where((df.Pregnancies > 10), "10+", df["PregnanciesCAT"])

# Hypoglycemia
df["Hypoglycemia"] = np.where(df.Glucose < 75, 1,0)

# Risk Score
df["IDAgeRiskScore"] = np.where(df.Age < 35, 0 , 20)
df["IDAgeRiskScore"] = np.where(df.Age > 49, 30 , df["IDAgeRiskScore"])
df["IDBMIRiskScore"] = np.where(df.BMI < 30 , 0, 10)
df["IDBMIRiskScore"] = np.where((df.BMI >= 45) & (df.BMI < 55) , 20, df["IDBMIRiskScore"])
df["IDBMIRiskScore"] = np.where((df.BMI >= 55), 30, df["IDBMIRiskScore"])


## Skin Thickness, BMI and Age
df["ST_BMI"] = df["SkinThickness"] / df["BMI"]
df["ST_BMI_Age"] = (df.BMI * df.SkinThickness ) / df.Age

df["New_SkinThickness"] = df["SkinThickness"] * 4

df.loc[(df['Age'] >= 21 ) & (df['Age'] <= 29) & (df["New_SkinThickness"] < 34), 'Body_Fat'] = "Underfat"
df.loc[(df['Age'] >= 30 ) & (df['Age'] <= 39) & (df["New_SkinThickness"] < 29), 'Body_Fat'] = "Underfat"
df.loc[(df['Age'] >= 40 ) & (df['Age'] <= 49) & (df["New_SkinThickness"] < 23), 'Body_Fat'] = "Underfat"
df.loc[(df['Age'] >= 50 ) & (df["New_SkinThickness"] < 20), 'Body_Fat'] = "Underfat"

df.loc[(df['Age'] >= 21 ) & (df['Age'] <= 29) & (df["New_SkinThickness"] >= 34) & (df["New_SkinThickness"] <= 79), 'Body_Fat'] = "Healthy"
df.loc[(df['Age'] >= 30 ) & (df['Age'] <= 39) & (df["New_SkinThickness"] >= 29) & (df["New_SkinThickness"] <= 73), 'Body_Fat'] = "Healthy"
df.loc[(df['Age'] >= 40 ) & (df['Age'] <= 49) & (df["New_SkinThickness"] >= 23) & (df["New_SkinThickness"] <= 59), 'Body_Fat'] = "Healthy"
df.loc[(df['Age'] >= 50 ) & (df["New_SkinThickness"] >= 20) & (df["New_SkinThickness"] <= 49), 'Body_Fat'] = "Healthy"

df.loc[(df['Age'] >= 21 ) & (df['Age'] <= 29) &( df["New_SkinThickness"] > 79) & (df["New_SkinThickness"] <= 120), 'Body_Fat'] = "OverFat"
df.loc[(df['Age'] >= 30) & (df['Age'] <= 39) & (df["New_SkinThickness"] > 73) & (df["New_SkinThickness"] <= 115), 'Body_Fat'] = "OverFat"
df.loc[(df['Age'] >= 40 ) & (df['Age'] <= 49) & (df["New_SkinThickness"] > 59) & (df["New_SkinThickness"] <= 95), 'Body_Fat'] = "OverFat"
df.loc[(df['Age'] >= 50 ) & (df["New_SkinThickness"] > 49) & (df["New_SkinThickness"] <= 77), 'Body_Fat'] = "OverFat"

df.loc[(df['Age'] >= 21 ) & (df['Age'] <= 29) & (df["New_SkinThickness"] > 120), 'Body_Fat'] = "Obese"
df.loc[(df['Age'] >= 30 ) & (df['Age'] <= 39) & (df["New_SkinThickness"] > 115) , 'Body_Fat'] = "Obese"
df.loc[(df['Age'] >= 40 ) & (df['Age'] <= 49) & (df["New_SkinThickness"] > 95) , 'Body_Fat'] = "Obese"
df.loc[(df['Age'] >= 50 )  & (df["New_SkinThickness"] > 77) , 'Body_Fat'] = "Obese"

df = df.drop(["New_SkinThickness"], axis=1)

## Glucose
df.loc[(df["Glucose"] < 140), "GlucoseCAT"] = "No_Risk"
df.loc[(df["Glucose"] >= 140), "GlucoseCAT"] = "Prediabetes"

<a id="section6"></a>
# 6. Numeric Variables 

In [ ]:
def num_plot(data, cat_length = 16, remove = ["Id"], hist_bins = 12, figsize = (20,4)):
    
    num_cols = [col for col in data.columns if data[col].dtypes != "O" 
                and len(data[col].unique()) >= cat_length]
    
    if len(remove) > 0:
        num_cols = list(set(num_cols).difference(remove))
            
    for i in num_cols:
        fig, axes = plt.subplots(1, 3, figsize = figsize)
        data.hist(str(i), bins = hist_bins, ax=axes[0])
        data.boxplot(str(i),  ax=axes[1], vert=False);
        try: 
            sns.kdeplot(np.array(data[str(i)]))
        except: ValueError
        
        axes[1].set_yticklabels([])
        axes[1].set_yticks([])
        axes[0].set_title(i + " | Histogram")
        axes[1].set_title(i + " | Boxplot")
        axes[2].set_title(i + " | Density")
        plt.show()
        
        
num_plot(df, cat_length = 16, hist_bins = 10, figsize = (20,4))

<a id="section7"></a>
# 7. Categoric Variables

In [ ]:
def cat_eda(data, cat_length, target = "Outcome"):  
    dataframe = data.copy()
    
    #if len(ordinal_variable) > 0:
    #    dataframe.drop(ordinal_variable, axis = 1, inplace = True)
        
    more_cat_cols = [col for col in dataframe.columns if len(dataframe[col].unique()) < cat_length]
    
    for i in more_cat_cols: 
        print(i, ":", len(dataframe[i].value_counts()), "Unique Category -", str(dataframe[i].dtype))
        print(pd.DataFrame({"COUNT": dataframe[i].value_counts(),
                            "RATIO": dataframe[i].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(i)[target].mean(),
                            "TARGET_COUNT": dataframe.groupby(i)[target].count(),
                            "TARGET_STD": dataframe.groupby(i)[target].std()}), end="\n\n\n")
    
    print("# DTYPES -----------------------------")
    print("Object Variables:",dataframe[more_cat_cols].select_dtypes("object").columns.tolist(), "\n")
    print("Integer Variables:",dataframe[more_cat_cols].select_dtypes("integer").columns.tolist(), "\n")
    print("Float Variables:",dataframe[more_cat_cols].select_dtypes("float").columns.tolist(), "\n")

cat_eda(df, cat_length=10)

<a id="section8"></a>
# 8. One-Hot Encoder

In [ ]:
# Transform Object
df["AGECAT"] = df["AGECAT"].astype(object)

cat_cols = [col for col in df.columns if df[col].dtypes == 'O']

def one_hot_encoder(dataframe, categorical_cols, nan_as_category=True):
    original_columns = list(dataframe.columns)
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, dummy_na=nan_as_category, drop_first=True)
    new_columns = [c for c in dataframe.columns if c not in original_columns]
    return dataframe, new_columns


df, new_cols_ohe = one_hot_encoder(df, cat_cols, nan_as_category=False)

<a id="section9"></a>
# 9. Model Comparision

In [ ]:
# All Models & Train Validation Scores
models = [('LR', LogisticRegression()),
          ("NBAYES", GaussianNB()),
          ('KNN', KNeighborsClassifier()),
          ('CART', DecisionTreeClassifier()),
          ("BAGGING", BaggingClassifier()),
          ('RF', RandomForestClassifier()),
          ('SVM', SVC(gamma='auto')),
          ("ADAB",AdaBoostClassifier()),
          ('GBM', GradientBoostingClassifier()),
          ("XGB", XGBClassifier()),
          ("LGBM", LGBMClassifier())]


y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=46)

# evaluate each model in turn
results = []
names = []

print("# Algorithm Comparison")
print("-----------------------------------------")
for name, model in models:
    kfold = KFold(n_splits=10, random_state=123456)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    
print("\n")    

# boxplot algorithm comparison
fig = plt.figure(figsize=(15, 10))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()